In [ ]:
from datetime import datetime
from getpass import getpass

rdm_url = 'https://bh.rdm.yzwlab.com/' # 'https://staging.rdm.example.com'
idp_name_1 = None # 'GakuNin RDM IdP'
idp_username_1 = None
idp_password_1 = None
idp_name_2 = None # 'GakuNin RDM IdP'
idp_username_2 = None
idp_password_2 = None
rdm_project_name = 'TEST-METADATA-{}'.format(datetime.now().strftime('%Y%m%d-%H%M%S'))
default_result_path = None
close_on_fail = False
transition_timeout = 10000
skip_failed_test = False
skip_erad_completion_test = False
exclude_notebooks = []

In [ ]:
if idp_username_1 is None:
    idp_username_1 = input(prompt=f'Username for {idp_name_1}')
if idp_password_1 is None:
    idp_password_1 = getpass(prompt=f'Password for {idp_username_1}@{idp_name_1}')
if idp_username_2 is None:
    idp_username_2 = input(prompt=f'Username for {idp_name_2}')
if idp_password_2 is None:
    idp_password_2 = getpass(prompt=f'Password for {idp_username_2}@{idp_name_2}')
(len(idp_username_1), len(idp_password_1), len(idp_username_2), len(idp_password_2))

In [ ]:
import tempfile

work_dir = tempfile.mkdtemp()
if default_result_path is None:
    default_result_path = work_dir
work_dir

# GakuNinRDM 総合テスト [Metadataアドオン]

- サブシステム名: ストレージ管理
- ページ/アドオン: Metadata
- 機能分類: メタデータ入力
- シナリオ名: *
- 用意するテストデータ: URL一覧、アカウント(既存ユーザー1: GRDM, 既存ユーザー2: GRDM)

In [ ]:
from datetime import datetime
import os
import papermill as pm
import traceback

def make_result_dir(base_path):
    result_dir = os.path.join(base_path, 'notebooks')
    os.makedirs(result_dir, exist_ok=True)
    return result_dir

def run_notebook(result_dir, base_notebook, optional_result_id=None, **optional_params):
    _, filename = os.path.split(base_notebook)
    
    # Check if notebook should be excluded
    if filename in exclude_notebooks:
        print(f'Skipping excluded notebook: {base_notebook}')
        return None
    
    result_id, _ = os.path.splitext(filename)
    if optional_result_id:
        result_id += optional_result_id
    result_notebook = os.path.join(result_dir, result_id + '.ipynb')
    result_path = os.path.join(result_dir, result_id)
    os.makedirs(result_path, exist_ok=True)
    params = dict(
        rdm_url=rdm_url,
        idp_name_1=idp_name_1,
        idp_username_1=idp_username_1,
        idp_password_1=idp_password_1,
        idp_name=idp_name_1,
        idp_username=idp_username_1,
        idp_password=idp_password_1,
        default_result_path=result_path,
        close_on_fail=True,
        transition_timeout=transition_timeout,
    )
    params.update(optional_params)
    try:
        pm.execute_notebook(
            base_notebook,
            result_notebook,
            parameters=params
        )
    except pm.PapermillExecutionError:
        if not skip_failed_test:
            raise
        print('失敗しました。テストは続行します...')
        traceback.print_exc()
    return result_notebook

result_dir = make_result_dir(default_result_path)
result_notebooks = []
result_dir

## 「アドオンの有効化」テストの実施

テスト「テスト手順-ストレージ共通-Metadataアドオン」を実施する。

In [ ]:
result_notebooks.append(run_notebook(
    result_dir, 'テスト手順-ストレージ共通-Metadataアドオン.ipynb',
    rdm_project_name=rdm_project_name + '-管理者',
))
result_notebooks[-1]

## 「非管理者での実行」テストの実施

テスト「テスト手順-Metadataアドオン-非管理者での実行」を実施する。

In [ ]:
result_notebooks.append(run_notebook(
    result_dir, 'テスト手順-Metadataアドオン-非管理者での実行.ipynb',
    rdm_project_name=rdm_project_name + '-非管理者',
    idp_name_2=idp_name_2,
    idp_username_2=idp_username_2,
    idp_password_2=idp_password_2,
))
result_notebooks[-1]

## 「複数メタデータの一括設定」テストの実施

テスト「テスト手順-Metadataアドオン-複数メタデータの一括設定」を実施する。

In [ ]:
result_notebooks.append(run_notebook(
    result_dir, 'テスト手順-Metadataアドオン-複数メタデータの一括設定.ipynb',
    rdm_project_name=rdm_project_name,
))
result_notebooks[-1]

## 「e-Rad情報を用いた自動補完」テストの実施

テスト「テスト手順-Metadataアドオン-e-Rad情報を用いた自動補完」を実施する。

In [ ]:
if not skip_erad_completion_test:
    result_notebooks.append(run_notebook(
        result_dir, 'テスト手順-Metadataアドオン-e-Rad情報を用いた自動補完.ipynb',
        rdm_project_name=rdm_project_name,
    ))
    result_notebooks[-1]
else:
    print('e-Rad自動補完テストはスキップします (skip_erad_completion_test=True)')

## 「KAKEN情報を用いた自動補完-自分の研究者番号」テストの実施

テスト「テスト手順-Metadataアドオン-KAKEN情報を用いた自動補完-自分の研究者番号」を実施する。

In [ ]:
if not skip_erad_completion_test:
    result_notebooks.append(run_notebook(
        result_dir, 'テスト手順-Metadataアドオン-KAKEN情報を用いた自動補完-自分の研究者番号.ipynb',
        rdm_project_name=rdm_project_name,
    ))
    result_notebooks[-1]
else:
    print('KAKEN情報自動補完テストはスキップします (skip_erad_completion_test=True)')

## 「KAKEN情報を用いた自動補完-共著者の研究者番号」テストの実施

テスト「テスト手順-Metadataアドオン-KAKEN情報を用いた自動補完-共著者の研究者番号」を実施する。

In [ ]:
if not skip_erad_completion_test:
    result_notebooks.append(run_notebook(
        result_dir, 'テスト手順-Metadataアドオン-KAKEN情報を用いた自動補完-共著者の研究者番号.ipynb',
        rdm_project_name=rdm_project_name,
        idp_name_2=idp_name_2,
        idp_username_2=idp_username_2,
        idp_password_2=idp_password_2,
    ))
    result_notebooks[-1]
else:
    print('KAKEN情報自動補完テストはスキップします (skip_erad_completion_test=True)')

## ウェブブラウザの新規プライベートウィンドウでGRDMトップページを表示する

GRDMトップページが表示されること

In [ ]:
import importlib
import pandas as pd

import scripts.playwright
importlib.reload(scripts.playwright)

from scripts.playwright import *
from scripts import grdm

await init_pw_context(close_on_fail=False, last_path=default_result_path)

In [ ]:
import time

async def _step(page):
    await page.goto(rdm_url)

    # 同意する をクリック
    await page.locator('//button[text() = "同意する"]').click()

    # 同意する が表示されなくなったことを確認
    await expect(page.locator('//button[text() = "同意する"]')).to_have_count(0, timeout=500)

await run_pw(_step)

## ログイン情報を用いてGakuNin RDMにログインする

(IdPに関するログイン情報が与えられた場合、)
GakuNin Embeded DSのプルダウンを展開し、IdPリストから指定されたIdPを選択する。その後、アカウントのID/Passwordを入力して「Login」ボタンを押下する。

(IdPが指定されていない場合、)
CASのログイン操作を実施する。

In [ ]:
import scripts.grdm
importlib.reload(scripts.grdm)

async def _step(page):
    await scripts.grdm.login(
        page, idp_name_1, idp_username_1, idp_password_1, transition_timeout=transition_timeout
    )

    # GRDMのボタンが表示されることを確認
    await expect(page.locator('//*[text() = "プロジェクト管理者"]')).to_be_visible(timeout=transition_timeout)

await run_pw(_step)

## ダッシュボードのプロジェクト一覧から指定されたプロジェクトをクリックする

In [ ]:
async def _step(page):
    await page.locator(f'//*[@data-test-dashboard-item-title and text()="{rdm_project_name}"]').click()        

    await expect(page.locator('//a[text() = "アドオン"]')).to_be_visible(timeout=transition_timeout)

await run_pw(_step)

## プロジェクトを削除する

In [ ]:
async def _step(page):
    await scripts.grdm.delete_project(page)
    
    await expect(page.locator('//*[text() = "プロジェクト管理者"]')).to_be_visible(timeout=transition_timeout)

await run_pw(_step)

終了処理を実施。

In [ ]:
await finish_pw_context()

In [ ]:
!rm -fr {work_dir}